# DiffEdit: 基于扩散模型的语义图像编辑

**论文**: [DiffEdit: Diffusion-based Semantic Image Editing with Mask Guidance](https://arxiv.org/abs/2210.11427) (Couairon et al., 2022)

## 核心思想

DiffEdit 提出了一种**零样本**文本引导图像编辑方法，无需用户手动提供掩码。整个流程分为三步：

1. **自动掩码生成 (Mask Generation)** — 对比 target prompt 和 reference prompt 条件下的噪声预测差异，自动识别需要编辑的区域
2. **DDIM 编码/反转 (DDIM Inversion)** — 将源图像通过 DDIM 反转映射回噪声空间，保留图像信息
3. **掩码 DDIM 解码 (Masked Decoding)** — 在去噪过程中，掩码内区域用目标文本引导去噪，掩码外区域保持编码时的 latent

**优势**：自动掩码 + 背景保持，无需手动标注，无需训练。

## 数学公式

### 1. 掩码生成

给定源图像 $x_0$，添加噪声得到 $x_t$，分别用 target prompt $P_{tgt}$ 和 reference prompt $P_{ref}$ 预测噪声：

$$M = \text{Binarize}\left( \frac{1}{N} \sum_{n=1}^{N} \left| \epsilon_\theta(x_t^{(n)}, P_{tgt}) - \epsilon_\theta(x_t^{(n)}, P_{ref}) \right| \right)$$

其中 $N$ 是采样次数，Binarize 是基于阈值的二值化操作。

### 2. DDIM 采样（前向去噪）

$$x_{t-1} = \sqrt{\bar\alpha_{t-1}} \underbrace{\left( \frac{x_t - \sqrt{1-\bar\alpha_t} \, \epsilon_\theta(x_t)}{\sqrt{\bar\alpha_t}} \right)}_{\text{predicted } x_0} + \sqrt{1 - \bar\alpha_{t-1}} \cdot \epsilon_\theta(x_t)$$

### 3. DDIM 反转（编码）

将去噪公式反转，从 $x_t$ 推导 $x_{t+1}$：

$$x_{t+1} = \sqrt{\bar\alpha_{t+1}} \underbrace{\left( \frac{x_t - \sqrt{1-\bar\alpha_t} \, \epsilon_\theta(x_t)}{\sqrt{\bar\alpha_t}} \right)}_{\text{predicted } x_0} + \sqrt{1 - \bar\alpha_{t+1}} \cdot \epsilon_\theta(x_t)$$

### 4. 掩码解码

每个去噪步 $t$：

$$\hat{x}_{t-1} = M \cdot \text{denoise}(\hat{x}_t, P_{tgt}) + (1-M) \cdot x_{t-1}^{\text{enc}}$$

其中 $x_{t-1}^{\text{enc}}$ 是 DDIM 反转过程中存储的对应时间步的 latent。

## 流水线示意图

```
源图像 x₀                 target: "a zebra"     reference: "a horse"
   │                          │                      │
   │                          ▼                      ▼
   │                    ┌─────────────────────────────────┐
   ├──(加噪)───────────►│  Step 1: 自动掩码生成            │
   │                    │  对比两种文本下的噪声预测差异       │
   │                    └──────────────┬──────────────────┘
   │                                   │ Mask M
   │                                   ▼
   │                    ┌─────────────────────────────────┐
   ├──(VAE编码)────────►│  Step 2: DDIM 反转 (编码)        │
   │                    │  x₀ → x₁ → x₂ → ... → x_T      │
   │                    │  存储所有中间 latent               │
   │                    └──────────────┬──────────────────┘
   │                                   │ {x_t} 序列
   │                                   ▼
   │                    ┌─────────────────────────────────┐
   │                    │  Step 3: 掩码 DDIM 解码          │
   │                    │  每步: mask内用target去噪          │
   │                    │        mask外用编码latent替换       │
   │                    └──────────────┬──────────────────┘
   │                                   │
   │                                   ▼
   │                              编辑后图像
   │                         (掩码内改变, 掩码外保持)
```

---
## 第二部分：环境搭建

In [ ]:
# 导入依赖
import logging
from pathlib import Path

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import torch
from PIL import Image

logging.disable(logging.WARNING)

# 中文字体配置
mpl.rcParams['font.sans-serif'] = ['Arial Unicode MS']  # macOS 自带，支持中文
mpl.rcParams['axes.unicode_minus'] = False               # 正常显示负号

# 设备检测
if torch.cuda.is_available():
    device = 'cuda'
elif torch.backends.mps.is_available():
    device = 'mps'
else:
    device = 'cpu'
print(f'使用设备: {device}')

torch.manual_seed(42)

In [ ]:
# 加载模型
from transformers import CLIPTextModel, CLIPTokenizer
from diffusers import AutoencoderKL, UNet2DConditionModel, DDIMScheduler

# CLIP 文本编码器
tokenizer = CLIPTokenizer.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16)
text_encoder = CLIPTextModel.from_pretrained("openai/clip-vit-large-patch14", torch_dtype=torch.float16).to(device)

# VAE
vae = AutoencoderKL.from_pretrained("stabilityai/sd-vae-ft-ema", torch_dtype=torch.float16).to(device)

# UNet
unet = UNet2DConditionModel.from_pretrained("CompVis/stable-diffusion-v1-4", subfolder="unet", torch_dtype=torch.float16).to(device)

# DDIM Scheduler — 关键: clip_sample=False，否则反转不精确
scheduler = DDIMScheduler(
    beta_start=0.00085,
    beta_end=0.012,
    beta_schedule="scaled_linear",
    clip_sample=False,        # 必须！不裁剪 pred_x0
    set_alpha_to_one=False,
    num_train_timesteps=1000,
)

print('模型加载完成！')

In [ ]:
# 工具函数

def text_enc(prompts, maxlen=None):
    """文本编码为 CLIP embeddings"""
    if maxlen is None: maxlen = tokenizer.model_max_length
    inp = tokenizer(prompts, padding="max_length", max_length=maxlen, truncation=True, return_tensors="pt")
    return text_encoder(inp.input_ids.to(device))[0].half()

def mk_img(t):
    """将张量转换为 PIL Image"""
    image = (t/2+0.5).clamp(0,1).detach().cpu().permute(1, 2, 0).numpy()
    return Image.fromarray((image*255).round().astype("uint8"))

def encode_img(image, generator=None):
    """将 PIL Image 编码为 VAE latent"""
    if isinstance(image, Image.Image):
        image = image.resize((512, 512))
        image = torch.tensor(np.array(image)).permute(2, 0, 1).unsqueeze(0).float() / 255.0
        image = (image * 2 - 1).to(device).half()
    with torch.no_grad():
        latent = vae.encode(image).latent_dist.sample(generator) * 0.18215
    return latent

def decode_latents(latents):
    """将 VAE latent 解码为 PIL Image"""
    with torch.no_grad():
        image = vae.decode(1 / 0.18215 * latents).sample
    return mk_img(image[0])

def show_images(images, titles=None, figsize=None, suptitle=None):
    """并排显示多张图片"""
    n = len(images)
    if figsize is None: figsize = (4*n, 4)
    fig, axes = plt.subplots(1, n, figsize=figsize)
    if n == 1: axes = [axes]
    for i, (ax, img) in enumerate(zip(axes, images)):
        ax.imshow(img)
        ax.axis('off')
        if titles: ax.set_title(titles[i], fontsize=12)
    if suptitle: fig.suptitle(suptitle, fontsize=14, y=1.02)
    plt.tight_layout()
    plt.show()

In [ ]:
# 用 Stable Diffusion 生成一张马的测试图片作为源图

def generate_image(prompt, num_steps=50, guidance_scale=7.5, seed=42):
    """使用 SD 从文本生成图片"""
    generator = torch.manual_seed(seed)
    
    # 文本编码
    text_emb = text_enc([prompt])
    uncond_emb = text_enc([""])
    emb = torch.cat([uncond_emb, text_emb])
    
    # 初始化随机噪声
    latents = torch.randn((1, 4, 64, 64), generator=generator, device=device, dtype=torch.float16)
    
    scheduler.set_timesteps(num_steps)
    latents = latents * scheduler.init_noise_sigma
    
    for t in scheduler.timesteps:
        latent_input = torch.cat([latents] * 2)
        latent_input = scheduler.scale_model_input(latent_input, t)
        
        with torch.no_grad():
            noise_pred = unet(latent_input, t, encoder_hidden_states=emb).sample
        
        # CFG
        noise_uncond, noise_text = noise_pred.chunk(2)
        noise_pred = noise_uncond + guidance_scale * (noise_text - noise_uncond)
        
        latents = scheduler.step(noise_pred, t, latents).prev_sample
    
    return latents, decode_latents(latents)

source_prompt = "a photograph of a horse on a grass field, high quality, 4k"
source_latents, source_image = generate_image(source_prompt)
show_images([source_image], titles=['源图像: 一匹马'])

---
## 第三部分：Step 1 — 自动掩码生成

### 原理

掩码生成的核心思想：**同一张有噪声的图片，在不同文本条件下预测的噪声是不同的**。

- 如果图像区域与两个 prompt 的语义差异无关（比如背景草地），两种条件下预测的噪声相似
- 如果图像区域是编辑目标（比如马 → 斑马），两种条件下预测的噪声差异很大

通过多次采样取平均，可以得到稳定的差异图，再二值化为掩码。

In [ ]:
def generate_mask(latents, target_prompt, reference_prompt, num_samples=10,
                  noise_level=0.5, threshold=0.5, num_inference_steps=50):
    """
    DiffEdit Step 1: 自动掩码生成
    
    对源图像 latent 添加噪声，比较 target 和 reference prompt 下的噪声预测差异，
    多次采样取平均后二值化得到掩码。
    
    Args:
        latents: 源图像的 VAE latent [1, 4, 64, 64]
        target_prompt: 目标文本 (如 "a zebra")
        reference_prompt: 参考文本 (如 "a horse")
        num_samples: 采样次数 N
        noise_level: 噪声水平 (0~1)，对应 scheduler 的时间步比例
        threshold: 二值化阈值
        num_inference_steps: scheduler 步数
    Returns:
        mask: 二值掩码 [1, 1, 64, 64]
        diff_map: 归一化差异图 [64, 64]
    """
    # 文本编码
    target_emb = text_enc([target_prompt])
    reference_emb = text_enc([reference_prompt])
    
    scheduler.set_timesteps(num_inference_steps)
    
    # 选择噪声水平对应的时间步
    t_idx = int(noise_level * num_inference_steps)
    t_idx = min(t_idx, len(scheduler.timesteps) - 1)
    t = scheduler.timesteps[-(t_idx + 1)]  # timesteps 是从大到小排列的
    
    diff_accumulator = torch.zeros(1, 4, 64, 64, device=device, dtype=torch.float32)
    
    for i in range(num_samples):
        # 添加随机噪声
        noise = torch.randn_like(latents)
        noisy_latents = scheduler.add_noise(latents, noise, t)
        
        with torch.no_grad():
            # target prompt 条件下的噪声预测
            noise_pred_target = unet(noisy_latents, t, encoder_hidden_states=target_emb).sample
            # reference prompt 条件下的噪声预测
            noise_pred_ref = unet(noisy_latents, t, encoder_hidden_states=reference_emb).sample
        
        # 累加差异的绝对值
        diff_accumulator += (noise_pred_target - noise_pred_ref).abs().float()
    
    # 平均
    diff_avg = diff_accumulator / num_samples
    
    # 对通道取平均 → [1, 1, 64, 64]
    diff_map = diff_avg.mean(dim=1, keepdim=True)
    
    # 归一化到 [0, 1]
    diff_map_norm = (diff_map - diff_map.min()) / (diff_map.max() - diff_map.min() + 1e-8)
    
    # 二值化
    mask = (diff_map_norm > threshold).float()
    
    return mask, diff_map_norm[0, 0].cpu().numpy()

In [ ]:
# 运行掩码生成
target_prompt = "a photograph of a zebra on a grass field"
reference_prompt = "a photograph of a horse on a grass field"

# 编码源图像
source_latent = encode_img(source_image)

mask, diff_map = generate_mask(
    source_latent, 
    target_prompt, 
    reference_prompt,
    num_samples=10,
    noise_level=0.5,
    threshold=0.5,
)
print(f'掩码形状: {mask.shape}, 掩码覆盖比例: {mask.mean().item():.2%}')

In [ ]:
# 可视化掩码
fig, axes = plt.subplots(1, 4, figsize=(18, 4))

# 原图
axes[0].imshow(source_image)
axes[0].set_title('原图', fontsize=12)
axes[0].axis('off')

# 差异热力图
im = axes[1].imshow(diff_map, cmap='hot', interpolation='nearest')
axes[1].set_title('噪声预测差异热力图', fontsize=12)
axes[1].axis('off')
plt.colorbar(im, ax=axes[1], fraction=0.046)

# 二值掩码
mask_np = mask[0, 0].cpu().numpy()
axes[2].imshow(mask_np, cmap='gray', interpolation='nearest')
axes[2].set_title('二值掩码 (阈值=0.5)', fontsize=12)
axes[2].axis('off')

# 掩码叠加在原图上
source_np = np.array(source_image.resize((64, 64)))
overlay = source_np.copy().astype(float)
mask_rgb = np.stack([mask_np * 255, mask_np * 50, mask_np * 50], axis=-1)
overlay = overlay * 0.6 + mask_rgb * 0.4
axes[3].imshow(overlay.clip(0, 255).astype(np.uint8))
axes[3].set_title('掩码叠加原图', fontsize=12)
axes[3].axis('off')

plt.suptitle('Step 1: 自动掩码生成结果 (horse → zebra)', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 阈值敏感性分析
thresholds = [0.3, 0.4, 0.5, 0.6, 0.7]
fig, axes = plt.subplots(1, len(thresholds), figsize=(4*len(thresholds), 4))

for ax, th in zip(axes, thresholds):
    mask_th = (torch.tensor(diff_map) > th).float().numpy()
    ax.imshow(mask_th, cmap='gray', interpolation='nearest')
    coverage = mask_th.mean()
    ax.set_title(f'阈值={th}\n覆盖率={coverage:.1%}', fontsize=11)
    ax.axis('off')

plt.suptitle('不同阈值下的掩码效果', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

---
## 第四部分：Step 2 — DDIM 编码（反转）

### 原理

DDIM 反转是 DDIM 采样的逆过程：

- **标准 DDIM**: $x_t \to x_{t-1}$（从噪声到干净图像）
- **DDIM 反转**: $x_t \to x_{t+1}$（从干净图像到噪声）

反转公式:

$$\hat{x}_0 = \frac{x_t - \sqrt{1 - \bar\alpha_t} \cdot \epsilon_\theta(x_t, t)}{\sqrt{\bar\alpha_t}}$$

$$x_{t+1} = \sqrt{\bar\alpha_{t+1}} \cdot \hat{x}_0 + \sqrt{1 - \bar\alpha_{t+1}} \cdot \epsilon_\theta(x_t, t)$$

**关键**: 反转时使用 `guidance_scale=1.0`（不用 CFG），因为 CFG 会放大误差，导致反转不可逆。

反转的目标是存储所有中间 latent $\{x_0, x_1, ..., x_T\}$，在 Step 3 中用于掩码外区域的保持。

In [ ]:
def ddim_inversion(latents, prompt, num_inference_steps=50, guidance_scale=1.0):
    """
    DiffEdit Step 2: DDIM 反转 (编码)
    
    将源图像 latent 通过 DDIM 反转映射到噪声空间，
    存储所有中间 latent 用于 Step 3 的掩码解码。
    
    Args:
        latents: 源图像的 VAE latent [1, 4, 64, 64]
        prompt: 文本提示（通常用源图像的描述）
        num_inference_steps: 推理步数
        guidance_scale: CFG 比例（反转时通常=1.0）
    Returns:
        all_latents: 所有中间 latent 的列表 [x_0, x_1, ..., x_T]
    """
    # 文本编码
    text_emb = text_enc([prompt])
    uncond_emb = text_enc([""])
    
    scheduler.set_timesteps(num_inference_steps)
    
    # 存储所有中间 latent（初始 latent 是 x_0）
    all_latents = [latents.clone()]
    
    # 反转时间步：从小到大（scheduler.timesteps 是从大到小，需要反转）
    reversed_timesteps = scheduler.timesteps.flip(0)
    
    for i, t in enumerate(reversed_timesteps):
        # 当前 latent
        x_t = latents
        
        if guidance_scale > 1.0:
            # CFG: 同时预测有条件和无条件
            latent_input = torch.cat([x_t] * 2)
            emb = torch.cat([uncond_emb, text_emb])
            with torch.no_grad():
                noise_pred = unet(latent_input, t, encoder_hidden_states=emb).sample
            noise_uncond, noise_text = noise_pred.chunk(2)
            noise_pred = noise_uncond + guidance_scale * (noise_text - noise_uncond)
        else:
            # 无 CFG (guidance_scale=1.0)
            with torch.no_grad():
                noise_pred = unet(x_t, t, encoder_hidden_states=text_emb).sample
        
        # 获取 alpha 值
        alpha_prod_t = scheduler.alphas_cumprod[t]
        
        # 计算下一个时间步
        if i < len(reversed_timesteps) - 1:
            t_next = reversed_timesteps[i + 1]
            alpha_prod_t_next = scheduler.alphas_cumprod[t_next]
        else:
            # 最后一步，使用最大时间步的 alpha
            alpha_prod_t_next = scheduler.alphas_cumprod[reversed_timesteps[-1]]
        
        # 预测 x_0
        pred_x0 = (x_t - torch.sqrt(1 - alpha_prod_t) * noise_pred) / torch.sqrt(alpha_prod_t)
        
        # 反转公式: x_{t+1} = sqrt(alpha_{t+1}) * pred_x0 + sqrt(1 - alpha_{t+1}) * eps
        latents = torch.sqrt(alpha_prod_t_next) * pred_x0 + torch.sqrt(1 - alpha_prod_t_next) * noise_pred
        
        all_latents.append(latents.clone())
    
    return all_latents

In [ ]:
# 运行 DDIM 反转
num_steps = 50
encode_ratio = 0.8  # 编码比率 r=0.8，只反转前 80% 的步数

# 设定步数为完整步数，反转时从第0步到第 r*num_steps 步
all_latents = ddim_inversion(source_latent, reference_prompt, num_inference_steps=num_steps)

start_step = int(encode_ratio * num_steps)  # 从第 40 步开始解码
print(f'反转完成! 共 {len(all_latents)} 个 latent (包括初始 x_0)')
print(f'编码比率 r={encode_ratio}, 解码起始步: {start_step}')

In [ ]:
# 验证反转质量: 用 DDIM 前向去噪来恢复原图 (guidance_scale=1.0)
def ddim_decode_simple(latents, prompt, num_inference_steps=50, guidance_scale=1.0):
    """简单的 DDIM 去噪（用于验证反转质量）"""
    text_emb = text_enc([prompt])
    uncond_emb = text_enc([""])
    
    scheduler.set_timesteps(num_inference_steps)
    
    for t in scheduler.timesteps:
        if guidance_scale > 1.0:
            latent_input = torch.cat([latents] * 2)
            emb = torch.cat([uncond_emb, text_emb])
            with torch.no_grad():
                noise_pred = unet(latent_input, t, encoder_hidden_states=emb).sample
            noise_uncond, noise_text = noise_pred.chunk(2)
            noise_pred = noise_uncond + guidance_scale * (noise_text - noise_uncond)
        else:
            with torch.no_grad():
                noise_pred = unet(latents, t, encoder_hidden_states=text_emb).sample
        latents = scheduler.step(noise_pred, t, latents).prev_sample
    
    return latents

# 从最后一个反转 latent 开始去噪 (g=1.0)
reconstructed_latent = ddim_decode_simple(all_latents[-1].clone(), reference_prompt, num_steps, guidance_scale=1.0)
reconstructed_image = decode_latents(reconstructed_latent)

show_images(
    [source_image, reconstructed_image],
    titles=['原图', 'DDIM 反转→去噪重建 (g=1.0)'],
    figsize=(10, 4)
)
print('如果反转正确，两张图应该非常接近。')

In [ ]:
# 可视化反转过程：展示 5 个等距中间 latent 的解码效果
num_show = 5
indices = np.linspace(0, len(all_latents)-1, num_show, dtype=int)
intermediate_imgs = [decode_latents(all_latents[idx]) for idx in indices]
titles = [f'Step {idx}/{len(all_latents)-1}' for idx in indices]

show_images(intermediate_imgs, titles=titles, figsize=(4*num_show, 4),
            suptitle='DDIM 反转过程中的中间 latent 解码')

---
## 第五部分：Step 3 — 掩码 DDIM 解码

### 原理

掩码解码在每个去噪步中执行：

1. 用**目标文本** (target prompt) + CFG 进行标准去噪，得到 `denoised`
2. 取 DDIM 反转中存储的对应时间步的编码 latent `encoded`
3. 用掩码融合：
   - **掩码内** (M=1)：使用去噪结果 → 被编辑为新内容
   - **掩码外** (M=0)：使用编码 latent → 保持原始内容

$$\hat{x}_{t-1} = M \cdot \text{denoised}_{t-1} + (1-M) \cdot x_{t-1}^{\text{encoded}}$$

In [ ]:
def diffedit_decode(all_latents, mask, target_prompt, num_inference_steps=50,
                    start_step=40, guidance_scale=7.5):
    """
    DiffEdit Step 3: 掩码 DDIM 解码
    
    从反转的中间 latent 开始，用目标文本去噪，同时用掩码保持背景。
    
    Args:
        all_latents: DDIM 反转得到的所有中间 latent [x_0, x_1, ..., x_T]
        mask: 二值掩码 [1, 1, 64, 64]，1 表示要编辑的区域
        target_prompt: 目标文本
        num_inference_steps: 推理步数
        start_step: 从第几步开始解码（对应编码比率）
        guidance_scale: CFG 比例
    Returns:
        latents: 最终解码的 latent
        intermediates: 中间结果列表（每 10 步存一张）
    """
    # 文本编码
    text_emb = text_enc([target_prompt])
    uncond_emb = text_enc([""])
    emb = torch.cat([uncond_emb, text_emb])
    
    scheduler.set_timesteps(num_inference_steps)
    
    # 从反转的第 start_step 个 latent 开始
    latents = all_latents[start_step].clone()
    
    # 只解码后 start_step 步
    decode_timesteps = scheduler.timesteps[-start_step:]
    
    # 将掩码扩展到 4 通道以匹配 latent
    mask_4ch = mask.expand(-1, 4, -1, -1).to(device).half()
    
    intermediates = []
    
    for i, t in enumerate(decode_timesteps):
        # CFG 去噪
        latent_input = torch.cat([latents] * 2)
        latent_input = scheduler.scale_model_input(latent_input, t)
        
        with torch.no_grad():
            noise_pred = unet(latent_input, t, encoder_hidden_states=emb).sample
        
        noise_uncond, noise_text = noise_pred.chunk(2)
        noise_pred = noise_uncond + guidance_scale * (noise_text - noise_uncond)
        
        # DDIM step
        denoised = scheduler.step(noise_pred, t, latents).prev_sample
        
        # 获取对应时间步的编码 latent
        # decode_timesteps[i] 对应 all_latents 中的第 (start_step - 1 - i) 个
        encoded_idx = start_step - 1 - i
        encoded_latent = all_latents[encoded_idx]
        
        # 掩码融合: mask 内用去噪结果，mask 外用编码 latent
        latents = mask_4ch * denoised + (1 - mask_4ch) * encoded_latent
        
        # 每 10 步存一张中间结果
        if (i + 1) % 10 == 0 or i == len(decode_timesteps) - 1:
            intermediates.append((i + 1, latents.clone()))
    
    return latents, intermediates

In [ ]:
# 运行 DiffEdit 解码
edited_latent, intermediates = diffedit_decode(
    all_latents, mask,
    target_prompt=target_prompt,
    num_inference_steps=num_steps,
    start_step=start_step,
    guidance_scale=7.5
)

edited_image = decode_latents(edited_latent)
print('DiffEdit 解码完成！')

In [ ]:
# 显示最终结果: 原图、掩码、编辑结果
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(source_image)
axes[0].set_title('原图 (horse)', fontsize=13)
axes[0].axis('off')

axes[1].imshow(mask[0, 0].cpu().numpy(), cmap='gray', interpolation='nearest')
axes[1].set_title('自动掩码', fontsize=13)
axes[1].axis('off')

axes[2].imshow(edited_image)
axes[2].set_title('DiffEdit 结果 (zebra)', fontsize=13)
axes[2].axis('off')

plt.suptitle('DiffEdit: horse → zebra', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 解码过程可视化
intermediate_imgs = [decode_latents(lat) for step, lat in intermediates]
titles = [f'Step {step}/{start_step}' for step, _ in intermediates]

show_images(intermediate_imgs, titles=titles, figsize=(4*len(intermediates), 4),
            suptitle='掩码 DDIM 解码过程')

---
## 第六部分：完整管线与实验

In [ ]:
def diffedit(source_image, target_prompt, reference_prompt,
             num_inference_steps=50, encode_ratio=0.8,
             guidance_scale=7.5, mask_threshold=0.5,
             num_mask_samples=10, noise_level=0.5):
    """
    DiffEdit 完整管线：自动掩码生成 + DDIM 反转 + 掩码解码
    
    Args:
        source_image: 源 PIL Image
        target_prompt: 目标文本描述
        reference_prompt: 参考文本描述（描述源图像）
        num_inference_steps: 推理步数
        encode_ratio: 编码比率 r (0~1)
        guidance_scale: CFG 比例（解码时使用）
        mask_threshold: 掩码二值化阈值
        num_mask_samples: 掩码生成采样次数
        noise_level: 掩码生成的噪声水平
    Returns:
        edited_image: 编辑后的 PIL Image
        mask: 二值掩码
        diff_map: 差异图
    """
    print('Step 1: 生成掩码...')
    source_latent = encode_img(source_image)
    mask, diff_map = generate_mask(
        source_latent, target_prompt, reference_prompt,
        num_samples=num_mask_samples, noise_level=noise_level,
        threshold=mask_threshold, num_inference_steps=num_inference_steps
    )
    print(f'  掩码覆盖率: {mask.mean().item():.2%}')
    
    print('Step 2: DDIM 反转...')
    all_latents = ddim_inversion(source_latent, reference_prompt, num_inference_steps)
    
    start_step = int(encode_ratio * num_inference_steps)
    print(f'  反转完成, 起始步: {start_step}')
    
    print('Step 3: 掩码解码...')
    edited_latent, _ = diffedit_decode(
        all_latents, mask, target_prompt,
        num_inference_steps=num_inference_steps,
        start_step=start_step,
        guidance_scale=guidance_scale
    )
    
    edited_image = decode_latents(edited_latent)
    print('完成！')
    
    return edited_image, mask, diff_map

In [ ]:
# 实验 1: 马 → 斑马（主实验）
edited_zebra, mask_zebra, diff_zebra = diffedit(
    source_image,
    target_prompt="a photograph of a zebra on a grass field",
    reference_prompt="a photograph of a horse on a grass field",
    encode_ratio=0.8,
    guidance_scale=7.5,
)

fig, axes = plt.subplots(1, 3, figsize=(15, 5))
for ax, img, title in zip(axes, 
    [source_image, mask_zebra[0,0].cpu().numpy(), edited_zebra],
    ['原图 (horse)', '自动掩码', '编辑结果 (zebra)']):
    if isinstance(img, np.ndarray) and img.ndim == 2:
        ax.imshow(img, cmap='gray')
    else:
        ax.imshow(img)
    ax.set_title(title, fontsize=13)
    ax.axis('off')
plt.suptitle('实验 1: horse → zebra', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

In [ ]:
# 实验 2: 更多编辑示例

# 先生成一张水果碗的源图
_, fruit_image = generate_image("a photograph of a bowl of fruits on a table, high quality", seed=123)

# 水果碗 → 花碗
edited_flowers, mask_flowers, _ = diffedit(
    fruit_image,
    target_prompt="a photograph of a bowl of flowers on a table",
    reference_prompt="a photograph of a bowl of fruits on a table",
    encode_ratio=0.8,
)

# 生成一张狗的源图
_, dog_image = generate_image("a photograph of a dog sitting in a park, high quality", seed=456)

# 狗 → 猫
edited_cat, mask_cat, _ = diffedit(
    dog_image,
    target_prompt="a photograph of a cat sitting in a park",
    reference_prompt="a photograph of a dog sitting in a park",
    encode_ratio=0.8,
)

# 展示结果
fig, axes = plt.subplots(2, 3, figsize=(15, 10))

for row, (src, msk, edited, label) in enumerate([
    (fruit_image, mask_flowers, edited_flowers, 'fruits → flowers'),
    (dog_image, mask_cat, edited_cat, 'dog → cat'),
]):
    axes[row, 0].imshow(src)
    axes[row, 0].set_title('原图', fontsize=12)
    axes[row, 0].axis('off')
    
    axes[row, 1].imshow(msk[0, 0].cpu().numpy(), cmap='gray')
    axes[row, 1].set_title('自动掩码', fontsize=12)
    axes[row, 1].axis('off')
    
    axes[row, 2].imshow(edited)
    axes[row, 2].set_title(f'编辑结果 ({label})', fontsize=12)
    axes[row, 2].axis('off')

plt.suptitle('实验 2: 更多编辑示例', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

### 编码比率消融

编码比率 $r$ 控制从噪声空间的哪个位置开始解码：

- **$r$ 小 (如 0.3)**：只反转到较低噪声水平，编辑能力弱，但背景保持好
- **$r$ 大 (如 0.9)**：反转到较高噪声水平，编辑能力强，但背景可能有变化
- **$r = 0.8$**：论文推荐的平衡点

直觉：$r$ 越大，给去噪过程的"自由度"越大，能做出更大的改变。

In [ ]:
# 编码比率消融实验
ratios = [0.3, 0.5, 0.7, 0.8, 0.9]
ablation_images = []

# 预先计算掩码和反转（只需做一次完整反转）
source_latent_abl = encode_img(source_image)
mask_abl, _ = generate_mask(
    source_latent_abl, target_prompt, reference_prompt,
    num_samples=10, noise_level=0.5, threshold=0.5
)
all_latents_abl = ddim_inversion(source_latent_abl, reference_prompt, num_steps)

for r in ratios:
    s = int(r * num_steps)
    edited_lat, _ = diffedit_decode(
        all_latents_abl, mask_abl, target_prompt,
        num_inference_steps=num_steps, start_step=s, guidance_scale=7.5
    )
    ablation_images.append(decode_latents(edited_lat))
    print(f'r={r} 完成')

titles = [f'r={r}' for r in ratios]
show_images(ablation_images, titles=titles, figsize=(4*len(ratios), 4),
            suptitle='编码比率 r 消融实验 (horse → zebra)')

In [ ]:
# 对比实验: DiffEdit vs 朴素 img2img

def naive_img2img(source_image, target_prompt, num_inference_steps=50,
                  encode_ratio=0.8, guidance_scale=7.5):
    """朴素 img2img: 直接加噪 + 去噪，无掩码保护"""
    source_latent = encode_img(source_image)
    
    text_emb = text_enc([target_prompt])
    uncond_emb = text_enc([""])
    emb = torch.cat([uncond_emb, text_emb])
    
    scheduler.set_timesteps(num_inference_steps)
    
    # 直接加噪到对应的时间步
    start_step = int(encode_ratio * num_inference_steps)
    t_start = scheduler.timesteps[-start_step]
    noise = torch.randn_like(source_latent)
    latents = scheduler.add_noise(source_latent, noise, t_start)
    
    # 从 start_step 开始去噪
    for t in scheduler.timesteps[-start_step:]:
        latent_input = torch.cat([latents] * 2)
        latent_input = scheduler.scale_model_input(latent_input, t)
        
        with torch.no_grad():
            noise_pred = unet(latent_input, t, encoder_hidden_states=emb).sample
        
        noise_uncond, noise_text = noise_pred.chunk(2)
        noise_pred = noise_uncond + guidance_scale * (noise_text - noise_uncond)
        
        latents = scheduler.step(noise_pred, t, latents).prev_sample
    
    return decode_latents(latents)

# 运行朴素 img2img
img2img_result = naive_img2img(source_image, target_prompt, encode_ratio=0.8)

# 对比展示
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

axes[0].imshow(source_image)
axes[0].set_title('原图', fontsize=13)
axes[0].axis('off')

axes[1].imshow(img2img_result)
axes[1].set_title('朴素 img2img\n(无掩码，背景改变!)', fontsize=12)
axes[1].axis('off')

axes[2].imshow(edited_zebra)
axes[2].set_title('DiffEdit\n(掩码保护，背景保持)', fontsize=12)
axes[2].axis('off')

plt.suptitle('DiffEdit vs 朴素 img2img 对比', fontsize=15, y=1.02)
plt.tight_layout()
plt.show()

---
## 第七部分：总结

### DiffEdit 三步法回顾

| 步骤 | 操作 | 关键参数 | 作用 |
|------|------|----------|------|
| Step 1 | 自动掩码生成 | `num_samples=10`, `threshold=0.5` | 自动识别编辑区域 |
| Step 2 | DDIM 反转 | `guidance_scale=1.0` | 保留源图信息 |
| Step 3 | 掩码解码 | `encode_ratio=0.8`, `guidance_scale=7.5` | 执行编辑+保持背景 |

### 优点
- **零样本**: 无需训练或微调
- **自动掩码**: 无需手动标注
- **背景保持**: 掩码外区域完美保留
- **灵活**: 通过编码比率控制编辑强度

### 局限性
- 掩码质量依赖 prompt 的准确性
- DDIM 反转在高 CFG 下不精确
- 编辑范围受限于简单的语义替换
- 计算成本较高（反转 + 多次采样生成掩码）

### 常见问题

**Q: 为什么 `clip_sample=False` 很重要？**

A: 如果 `clip_sample=True`，scheduler 会裁剪 predicted $x_0$ 到 $[-1, 1]$，这在生成时是合理的（避免 latent 溢出），但在反转时会引入不可逆的信息损失，导致编码-解码不匹配。

**Q: VAE 缩放因子 0.18215 是什么？**

A: Stable Diffusion 训练时，VAE latent 被缩放使其标准差约为 1。0.18215 是训练集上统计得到的缩放因子。编码时乘以它，解码时除以它。

**Q: 掩码分辨率为什么是 64×64？**

A: 因为掩码在 latent 空间计算。VAE 的下采样倍率为 8x，所以 512×512 图像对应 64×64 latent。

**Q: MPS (Apple Silicon) 兼容性？**

A: MPS 支持基本推理，但可能在某些操作上有精度差异。如果结果异常，可以尝试将关键计算转到 float32。

In [ ]:
# 进阶: 掩码高斯模糊（平滑掩码边缘，减少编辑区域的突兀感）
from scipy.ndimage import gaussian_filter

def smooth_mask(mask, sigma=2.0, threshold=0.5):
    """对掩码进行高斯模糊，使边缘更平滑"""
    mask_np = mask[0, 0].cpu().numpy().astype(float)
    smoothed = gaussian_filter(mask_np, sigma=sigma)
    # 可选: 重新二值化（如果需要硬边缘）或保持软掩码
    return torch.tensor(smoothed, device=device, dtype=torch.float16).unsqueeze(0).unsqueeze(0)

# 对比硬掩码 vs 软掩码
soft_mask = smooth_mask(mask, sigma=2.0)

# 用软掩码重新解码
edited_soft, _ = diffedit_decode(
    all_latents, soft_mask, target_prompt,
    num_inference_steps=num_steps, start_step=start_step, guidance_scale=7.5
)
edited_soft_img = decode_latents(edited_soft)

fig, axes = plt.subplots(1, 4, figsize=(20, 5))

axes[0].imshow(mask[0, 0].cpu().numpy(), cmap='gray')
axes[0].set_title('硬掩码', fontsize=12)
axes[0].axis('off')

axes[1].imshow(soft_mask[0, 0].cpu().numpy(), cmap='gray')
axes[1].set_title('软掩码 (σ=2.0)', fontsize=12)
axes[1].axis('off')

axes[2].imshow(edited_zebra)
axes[2].set_title('硬掩码编辑结果', fontsize=12)
axes[2].axis('off')

axes[3].imshow(edited_soft_img)
axes[3].set_title('软掩码编辑结果', fontsize=12)
axes[3].axis('off')

plt.suptitle('硬掩码 vs 软掩码（高斯模糊）', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()

### 参考文献

1. **DiffEdit**: Couairon et al., "DiffEdit: Diffusion-based Semantic Image Editing with Mask Guidance", ICLR 2023. [arXiv:2210.11427](https://arxiv.org/abs/2210.11427)
2. **DDIM**: Song et al., "Denoising Diffusion Implicit Models", ICLR 2021. [arXiv:2010.02502](https://arxiv.org/abs/2010.02502)
3. **Latent Diffusion Models**: Rombach et al., "High-Resolution Image Synthesis with Latent Diffusion Models", CVPR 2022. [arXiv:2112.10752](https://arxiv.org/abs/2112.10752)
4. **Classifier-Free Guidance**: Ho & Salimans, "Classifier-Free Diffusion Guidance", NeurIPS 2021 Workshop. [arXiv:2207.12598](https://arxiv.org/abs/2207.12598)